In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import h5py
import time
import glob
import math
import shutil
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, ZeroPadding2D, Cropping2D, BatchNormalization, Flatten, Bidirectional, Dropout, Reshape,InputSpec
from keras import regularizers
from tensorflow.python.keras.engine.base_layer import Layer
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping, TensorBoard, CSVLogger
from keras import optimizers
from keras import metrics
from keras.utils import plot_model
import random
from math import *
import json
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from keras import backend as K
from sklearn.preprocessing import StandardScaler

from matplotlib.pyplot import savefig
from tensorflow.python.ops import math_ops
from sklearn.cluster import KMeans, AgglomerativeClustering, FeatureAgglomeration, DBSCAN, Birch
from sklearn.mixture import GaussianMixture
import tensorflow as tf
from sklearn.metrics.pairwise import pairwise_distances
import struct
import matplotlib
from matplotlib.ticker import StrMethodFormatter
from matplotlib.ticker import FormatStrFormatter
import datetime
import sys

length  of input 
length of latent representation

In [4]:
train_dataname = './data_processed.npy'
data_processed = np.load(train_dataname)

In [9]:
h,w,c = np.shape(data_processed)
data = np.empty((h,w,c,1))
for i in range(c):
    data[:,:,i,0] = data_processed[:,:,i]
data_input = Input(shape=(h, w, 1))

## auto encoder part

In [11]:
depth      = 8
strides    = 1
activation = 'relu'
# ReLU initial weights
kernel_initializer='glorot_uniform'
latent_dim = 14

In [13]:
# encoder
layer1 = Conv2D(depth*2**0, (5,5), strides=strides, activation=activation, kernel_initializer=kernel_initializer, padding='same') (img_input)
layer2 = Conv2D(depth*2**1, (5,5), strides=strides, activation=activation, kernel_initializer=kernel_initializer, padding='same') (e)
layer3 = Conv2D(depth*2**2, (3,3), strides=strides, activation=activation, kernel_initializer=kernel_initializer, padding='same') (e)
layer4 = Conv2D(depth*2**3, (3,3), strides=strides, activation=activation, kernel_initializer=kernel_initializer, padding='same')(e)

In [ ]:
# decode
decoder_layer1 = Dense(np.prod(shape_before_flattening[1:]), activation='relu')(encoded)
decoder_layer2 = Reshape(shape_before_flattening[1:])(d)
decoder_layer3 = Conv2DTranspose(depth*2**2, (3,3), strides=strides, activation=activation, kernel_initializer=kernel_initializer, padding='same')(d)
decoder_layer4= Conv2DTranspose(depth*2**1, (5,5), strides=strides, activation=activation, kernel_initializer=kernel_initializer, padding='same')(d)
decoder_layer5 = Conv2DTranspose(depth*2**0, (5,5), strides=strides, activation=activation, kernel_initializer=kernel_initializer, padding='same')(d)

In [ ]:
encoder = Model(inputs=img_input, outputs=encoded, name='encoder')
autoencoder = Model(inputs=img_input, outputs=decoded, name='autoencoder')

In [ ]:
learning_rate = 0.0001 
n_epochs = 500  
batch_size = 1024 

logger_fname = 'HistoryLog_LearningCurve.csv'.format(date_name)
csv_logger = CSVLogger(logger_fname)


early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1,
                           mode='min', restore_best_weights=True)

optim = tf.keras.optimizers.Adam(lr=learning_rate)             
loss = 'mse'                              

encoder.compile(loss=loss,optimizer=optim)
autoencoder.compile(loss=loss,
                  optimizer=optim,
                  metrics=[metrics.mae])


In [ ]:
autoencoder.fit(data, data, batch_size=batch_size, epochs=n_epochs, callbacks=[csv_logger, early_stop])

## GMM fitting part

- get latent space

In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.contrib.distributions import MultivariateNormalFullCovariance
MVNFC = MultivariateNormalFullCovariance


def gmm(mu, cov, tau, sx_proj, n_clusters=None, gmm_type='natural',
        trainable=True, cov_diag=None):
    """Gaussian mixture clustering."""
    if trainable is True:
        log_tau = tf.log(tf.nn.softmax(tau))
        eps = tf.eye(2)
        d = cov_diag
        gm = [MVNFC(mu[c], tf.nn.elu(d[c]) * np.eye(2) +
                    tf.matmul(cov[c], tf.transpose(cov[c])) + eps)
              for c in range(n_clusters)]
    else:
        n_clusters = cov.get_shape().as_list()[0]
        log_tau = tf.log(tau)
        gm = [MVNFC(mu[c], cov[c]) for c in range(n_clusters)]
    log_p = [gm[c].log_prob(sx_proj) for c in range(n_clusters)]
    cat = tf.stack([log_tau[c] + log_p[c] for c in range(n_clusters)], 1)

    # Choose you loss
    if gmm_type == 'natural':
        q = tf.reduce_logsumexp(cat, axis=1)
        loss = - tf.reduce_mean(q)
    else:
        y = tf.argmax(cat, axis=1)
        y_un, _ = tf.unique(y)
        q = tf.reduce_sum(tf.stop_gradient(tf.nn.softmax(cat)) * cat, 1)
        loss = tf.map_fn(
            lambda c: tf.reduce_sum(
                q * tf.cast(tf.equal(y, tf.ones_like(y) * c),
                            tf.float32)) /
            tf.reduce_sum(
                tf.cast(tf.equal(y, tf.ones_like(y) * c), tf.float32)),
            y_un, dtype=tf.float32) * 1e3

    return loss, cat

## KMEANS fitting part

In [ ]:
for i in np.arange(2, 10, 1):
    kmeans_model_test = KMeans(n_clusters=i).fit(enc_train)
    inertia.append(kmeans_model_test.inertia_)

In [ ]:
kmeans = KMeans(n_clusters=n_clusters, n_init=100) # run kmeans with n_clusters, run 100 initializations to ensure accuracy

labels = kmeans.fit_predict(enc_train)             # get initial assignments

labels_last = np.copy(labels)  